In [ ]:
!pip install langchain langchain-community #reposnsible for doc to pdf vice versa that is help to get tool
!pip install faiss-cpu#facebook ai similarity search()
!pip install pypdf python-docx#if apply text internally it will be converted into pdf (resolution)
!pip install sentence-transformers#here we put text form it should be in sentence onlyyy
!pip install transformers

In [ ]:
from google.colab import files #suprivificial layer of google cloud
uploaded = files.upload()

file_path = list(uploaded.keys())[0] #we dont konow what will be there(text emoji others)
print("uploaded:", file_path)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter #becuse to get similarity so it is broken down into too depth



#Pick loader based on file types

if file_path.endswith('.pdf'):
  loader = PyPDFLoader(file_path)
elif file_path.endswith('.docx') or file_path.endswith('.doc'):   #loader only will be there becuse answer will b for only one particulaer variable

  loader = Docx2txtLoader(file_path)
else:
  loader = TextLoader(file_path)

docs = loader.load()

#Split into small chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100) #ex 100 chunk will be 1 overlap...overlap depends on cunk size
documents = splitter.split_documents(docs)
print(f"Total Chunks: {len(documents)}")

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings   #adding (collecting) all small models which we are embinding from the hugging face
from langchain_community.vectorstores import FAISS #creating a storage whuch is aldready in vector that is boght by FISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
from transformers import pipeline #CREATING A function here ai should work
from langchain_community.llms import HuggingFacePipeline #llms from evry tool it should be undertood ....may be in pdf reading or anything
flan_pipeline=pipeline(         #change to flan-t5 small fir even faster.....512 if we give more it will be halacinutaed
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=512
)
llm=HuggingFacePipeline(pipeline=flan_pipeline)

In [ ]:
from langchain.chains import RetrievalQA #to make the process fast

qa = RetrievalQA.from_chain_type(    #serach best 3 answers
     #kind a assinging the value
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k":3}),
    chain_type="stuff" #carring thos sequence of documents in llm...here it is 3 answers(documents)
)

query = "give me more summary of the documents" #testing lang and query
print(qa.run(query))



In [ ]:
while True:
  q = input("aska a question(or 'exit'):")
  if q.lower() == "exit":
    break

  print("answers:",qa.run(q))
